In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np

In [ ]:
from josie.bc import Dirichlet, Neumann, NeumannDirichlet
from josie.geom import Line
from josie.solver.euler import Q, EulerSolver, PerfectGas, Rusanov
from josie.mesh import Mesh, SimpleCell

left = Line([0, 0], [0, 1])
bottom = Line([0, 0], [1, 0])
right = Line([1, 0], [1, 1])
top = Line([0, 1], [1, 1])

eos = PerfectGas(gamma=1.4)

# Params
JET_CENTER = 0.5
JET_RADIUS = 0.05

# Inlet
U_JET = 1
V_JET = 0
RHO_JET = 2
P_JET = 10

RHOe_JET = eos.rhoe(RHO_JET, P_JET)
E_JET = RHOe_JET/RHO_JET + 0.5 * (U_JET**2 + V_JET**2)
C_JET = eos.sound_velocity(RHO_JET, P_JET)
print(f"Mach: {U_JET/C_JET}")
Q_JET = Q(RHO_JET, RHO_JET*U_JET, RHO_JET*V_JET, RHO_JET*E_JET, RHOe_JET, U_JET, V_JET, P_JET, C_JET)

# Field conditions at init
U_INIT = 0.01
V_INIT = 0
RHO_INIT = 1
P_INIT = 5
RHOe_INIT = eos.rhoe(RHO_INIT, P_INIT)
E_INIT = RHOe_INIT/RHO_INIT + 0.5 * (U_INIT**2 + V_INIT**2)
C_INIT = eos.sound_velocity(RHO_INIT, P_INIT)

print(f"Mach: {U_INIT/C_INIT}")

Q_INIT = Q(RHO_INIT, RHO_INIT*U_INIT, RHO_INIT*V_INIT, RHO_INIT*E_INIT, RHOe_INIT, 
           U_INIT, V_INIT, P_INIT, C_INIT)

# Neumann
dQ = Q.zeros()

def partition_fun(centroids: np.ndarray):
    yc = centroids[:, 1]
    
    # Partition cells of the inlet
    idx = np.where((yc - JET_CENTER)**2 < JET_RADIUS**2)
    
    print(idx)
    return idx


# Assign BC to boundaries
# left.bc = NeumannDirichlet(dirichlet_value=Q_JET, neumann_value=dQ, partition_fun=partition_fun)
left.bc = Dirichlet(Q_JET)
top.bc = Dirichlet(Q_INIT)
right.bc = Dirichlet(Q_INIT)
bottom.bc = Dirichlet(Q_INIT)

mesh = Mesh(left, bottom, right, top, SimpleCell)
mesh.interpolate(50, 50)
mesh.generate()
mesh.write('mesh.xdmf')
# mesh.plot()

In [ ]:
# Solver
solver = EulerSolver(mesh, eos)



def init_fun(solver): 
    solver.values[:, :, :] = Q_INIT
    

solver.init(init_fun)

In [ ]:
print(solver.left_ghost[:, 0])
print(solver.mesh.centroids[0, :, 1])

In [ ]:
solver.plot()
solver.show('rhoU')

In [ ]:
final_time = 0.25
t = 0
CFL = 0.2
scheme = Rusanov()

while t <= final_time:
    dt = scheme.CFL(
        solver.values, 
        solver.mesh.volumes, 
        solver.mesh.normals,
        solver.mesh.surfaces,
        CFL,
    )
    solver.step(dt, scheme)
    t += dt
    print(t)
    solver.save(t, 'euler.xdmf')


del solver._writer

In [ ]:
del solver._writer